In [1]:
import gym
from collections import deque
import numpy as np
import random
import torch
import pdb
import torch.optim as optim 
from torch import nn

In [2]:
env = gym.make('MountainCar-v0')

In [3]:
class Brain(nn.Module):
    def __init__(self,state_dim,action_space):
        super(Brain, self).__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(state_dim, 12),
            nn.ReLU(),
            nn.Linear(12, 24),
            nn.ReLU(),
            nn.Linear(24, 48),
            nn.ReLU(),
            nn.Linear(48, action_space),
        )

    def forward(self, x):
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
class Agent:
    def __init__(self,env):
        self.env=env
        self.gamma=0.96
        self.epsilon = 0.9
        self.learingRate=0.001
        self.replayBuffer=deque(maxlen=10000)
        self.trainNetwork  = Brain(2,3)
        self.targetNetwork = Brain(2,3)
        self.episodeNum=200
        self.iterationNum=400 
        self.numPickFromBuffer=32
        self.rewardWindow=[]
        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(self.trainNetwork.parameters(), lr=0.0001)
        
    def bestAction(self,state):
        if np.random.rand() < self.epsilon:
            action = np.random.randint(0, 3)
        else:
            state = torch.tensor(state,dtype=torch.float32)
            action=torch.argmax(self.trainNetwork(state))
        return int(action)
        
    def trainBrain(self):
        if len(self.replayBuffer) < self.numPickFromBuffer:
            return 
        
        currents = []
        news = []
        rewards = []
        actions = []
        
        samples = random.sample(self.replayBuffer,self.numPickFromBuffer)
        
        for curr in samples:
            currents.append(curr[0])
        currents = np.array(currents)
        
        for new in samples:
            news.append(new[3])
        news = np.array(news)
        
        for reward in samples:
            rewards.append(reward[2])
        rewards = np.array(rewards)
        
        for action in samples:
            actions.append(action[1])
            
        
        
        next_Q = self.targetNetwork(torch.tensor(news,dtype=torch.float32))
        current_q = self.trainNetwork(torch.tensor(currents,dtype=torch.float32))
        
        new_q = rewards + self.gamma*np.max(next_Q.detach().numpy(),axis=1)
        pdb.set_trace()
        
        for i in range(len(actions)):
            current_q[i,actions[i]] = new_q[i]
            
#         pdb.set_trace()
        x = torch.tensor(currents,dtype=torch.float32)
        y = torch.tensor(current_q,dtype=torch.float32)
        
        out = self.trainNetwork(x)
        loss = self.criterion(out, y)
#         print(loss)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        
    def play(self):
        current_state = self.env.reset()
        while True:
            current_state = torch.tensor(current_state,dtype=torch.float32)
            action=torch.argmax(self.trainNetwork(current_state))
            new_state, reward, done, _ = env.step(int(action))
            current_state = new_state
            self.env.render()
            if done:
                current_state = self.env.reset()
        
        
    def main(self):
        for it in range(self.iterationNum):
            current_state = self.env.reset()
            for eps in range(self.episodeNum):
                action = self.bestAction(current_state)
#                 if it >50:
#                     print(action)
                new_state, reward, done, _ = self.env.step(action)
                if reward == 0:
                    print(f'success at iteration {it} and episode {eps}')
                self.replayBuffer.append([current_state, action, reward, new_state, done])
                self.trainBrain()
                current_state = new_state  
#                 print(f'current state is {current_state[0]}')
#                 if done:
#                     break
                    
                    
            self.targetNetwork.load_state_dict(self.trainNetwork.state_dict())


                
        
        

In [5]:
agent = Agent(env)

In [6]:
agent.main()

> <ipython-input-4-b5dca2b897cb>(59)trainBrain()
     57         pdb.set_trace()
     58 
---> 59         for i in range(len(actions)):
     60             current_q[i,actions[i]] = new_q[i]
     61 

ipdb> new_q.shape
(32,)
ipdb> q


BdbQuit: 

In [15]:
# agent.play()

In [22]:
current_state = env.reset()
print(current_state)
print('-------------------------')
while True:
#     current_state = torch.tensor(current_state,dtype=torch.float32)
#     action=torch.argmax(agent.targetNetwork(current_state))  
    action = np.random.randint(0,3)
    new_state, reward, done, _ = env.step(action)
#     rep.append([current_state, action, reward, new_state, done])
    current_state = new_state
#     print(current_state)
#     env.render()
    if done:
        if reward == 0:
            print('win')
            break
env.close()

[-0.55402311  0.        ]
-------------------------


KeyboardInterrupt: 

In [37]:
inp = torch.tensor(np.random.rand(10,2),dtype=torch.float32)
tar = torch.tensor(np.random.rand(10,3),dtype=torch.float32)
criterion = nn.MSELoss()


In [38]:
out = dqn(inp)

In [39]:
out.shape

torch.Size([10, 3])

In [46]:
np.max(out.detach().numpy(),axis=1)

array([0.27805674, 0.33564025, 0.37856692, 0.31040233, 0.38727546,
       0.25115293, 0.42456454, 0.35666302, 0.25552255, 0.42150941],
      dtype=float32)

In [40]:
loss = criterion(out, tar)

In [61]:
d = deque(maxlen=50)

In [62]:
current_state = env.reset()
for eps in range(150):
    action = np.random.randint(0,3)
    new_state, reward, done, _ = env.step(action)
    d.append([current_state, action, reward, new_state, done])
    current_state = new_state

In [63]:
currents = []
news = []
rewards = []
actions = []

samples = random.sample(d,3)

for curr in samples:
    currents.append(curr[0])
currents = np.array(currents)

for new in samples:
    news.append(new[3])
news = np.array(news)

for reward in samples:
    rewards.append(reward[2])
rewards = np.array(rewards)

for action in samples:
    actions.append(action[1])

In [64]:
currents

array([[-0.35872627,  0.0041703 ],
       [-0.39517037, -0.00817855],
       [-0.72402762, -0.0013956 ]])

In [65]:
news

array([[-3.56742706e-01,  1.98356053e-03],
       [-4.05288482e-01, -1.01181093e-02],
       [-7.24008959e-01,  1.86583322e-05]])

In [66]:
actions

[0, 0, 1]

In [67]:
rewards

array([-1., -1., -1.])

In [68]:
next_Q = agent.targetNetwork(torch.tensor(news,dtype=torch.float32))
current_q = agent.trainNetwork(torch.tensor(currents,dtype=torch.float32))

new_q = rewards + 1*np.max(next_Q.detach().numpy(),axis=1)
print(current_q)
print('-----------------------------')

for i in range(len(actions)):
    current_q[i,actions[i]] = new_q[i]
print(current_q)

tensor([[-9.9953, -9.9969, -9.9971],
        [-9.9950, -9.9969, -9.9970],
        [-9.9953, -9.9971, -9.9973]], grad_fn=<AddmmBackward>)
-----------------------------
tensor([[-10.9952,  -9.9969,  -9.9971],
        [-10.9949,  -9.9969,  -9.9970],
        [ -9.9953, -10.9954,  -9.9973]], grad_fn=<CopySlices>)


In [101]:
env = gym.make('CartPole-v0')

In [151]:
current_state = env.reset()
while True:
    action = 1
    new_state, reward, done, _ = env.step(action)
    print(reward)
    d.append([current_state, action, reward, new_state, done])
    current_state = new_state
#     if done:
#         break
    env.render()
env.close()

1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


KeyboardInterrupt: 

In [103]:
env.observation_space

Box(4,)

In [124]:
env.close()